# Joining Tables in SQL #
## Prerequisites ##
- For this tutorial you should already have completed the 1_intro_to_sql tutorial. Your schools.db file should contain the students table with multiple students in it, with a sequence that automatically generates student ids. 

## Primary and Foreign Keys ##

Primary keys are unique identifiers for each row in a table. They are used to ensure that each row in a table is unique. Foreign keys are used to link two tables together. They are used to ensure that the value in a column in one table is present in another table. 

We have already created a `students` table. For our next move we will create an `awards` table. This table will contain information about awards that students have won. Each award can only be given to one student, so we will need to link the `awards` table to the `students` table. This saves us from duplicating the relevant information about a student in the `awards` table.

**In the code below we will:**
- create the `awards` table with the following columns:
    - `award_id` - a unique identifier for each award
    - `student_id` - the id of the student who won the award. This is a foreign key that links to the `students` table
    - `award_name` - the name of the award
    - `award_date` - the date the award was won (defaulting to the current date)


In [1]:
# Importing relevant libraries. This needs to run every time the script is run.
import duckdb

In [18]:

# Create the awards table in the database. We are now using CREATE OR REPLACE TABLE to avoid errors if the table already exists.
# This also means if you mess your awards table up, you can just run this script again to reset it.
# We add a sequence to the award_id column to automatically generate unique IDs.

with duckdb.connect('../data/school.db') as con:
    # create the sequence first. This will automatically generate unique IDs for the awards table.
    con.sql('''
        CREATE OR REPLACE SEQUENCE award_id_seq;
    ''')
    
    con.sql('''
            CREATE OR REPLACE TABLE awards (
            award_id INTEGER PRIMARY KEY DEFAULT nextval('award_id_seq'),
            student_id INTEGER,
            award_name VARCHAR,
            award_date DATE default CURRENT_DATE,
            FOREIGN KEY (student_id) REFERENCES students(student_id)
        );
    ''')
    
    con.table('awards').show()

┌──────────┬────────────┬────────────┬────────────┐
│ award_id │ student_id │ award_name │ award_date │
│  int32   │   int32    │  varchar   │    date    │
├──────────┴────────────┴────────────┴────────────┤
│                     0 rows                      │
└─────────────────────────────────────────────────┘



### What does the foreign key do? ###
The foreign key in the `awards` table will ensure that the `student_id` in the `awards` table is present in the `students` table. This means that we can't add an award for a student that doesn't exist in the `students` table. 

Equally importantly, it means that any time we want to know any more about which student got the award, we can use the `student_id` to look up the student in the `students` table. 

**In the code below, we are:**
- Checking which students are in the `students` table - we need to know their `student_id` to add an award for them
- Adding rows to the `awards` table. We need to add the `student_id` of the student who won the award and the `award_name`. The default values for `award_id` and `award_date` will be automatically generated.  


In [7]:
# Verify which students currently exist in the database. We can now only create rows in the awards table if 
# the student_id already exists in the students table.

with duckdb.connect('../data/school.db') as con:
    con.sql('SELECT * FROM students').show()

┌────────────┬────────────┬───────────┬───────┬────────────┐
│ student_id │ first_name │ last_name │  age  │ year_level │
│   int32    │  varchar   │  varchar  │ int32 │   int32    │
├────────────┼────────────┼───────────┼───────┼────────────┤
│          1 │ Roger      │ Rabbit    │    18 │         11 │
│          2 │ Jessica    │ Rabbit    │    17 │         10 │
│          3 │ Bugs       │ Bunny     │    13 │          7 │
│          4 │ Daffy      │ Duck      │    10 │          9 │
│        100 │ Charlie    │ Bucket    │    17 │         11 │
└────────────┴────────────┴───────────┴───────┴────────────┘



In [19]:
# Add new rows to the awards table. We are using INSERT INTO to add new rows to the table.
with duckdb.connect('../data/school.db') as con:
    # Insert a row. 
    con.sql(''' INSERT INTO awards (student_id, award_name)
            VALUES (1, '2024 English Award'); ''')
    
    # Insert a row with a specific award date.
    con.sql(''' INSERT INTO awards (student_id, award_name, award_date)
            VALUES (100, '2024 Maths Award', '2024-06-01'); ''')
    
    # Insert a second row for the same student.
    con.sql(''' INSERT INTO awards (student_id, award_name)
            VALUES (100, '2024 School Spirit Award'); ''')
    # Show the awards table.
    con.sql(''' SELECT * FROM awards; ''').show()

┌──────────┬────────────┬──────────────────────────┬────────────┐
│ award_id │ student_id │        award_name        │ award_date │
│  int32   │   int32    │         varchar          │    date    │
├──────────┼────────────┼──────────────────────────┼────────────┤
│        1 │          1 │ 2024 English Award       │ 2024-09-20 │
│        2 │        100 │ 2024 Maths Award         │ 2024-06-01 │
│        3 │        100 │ 2024 School Spirit Award │ 2024-09-20 │
└──────────┴────────────┴──────────────────────────┴────────────┘



# Joining the tables with a query (Inner Join) #
Now that we have two tables, we can join them together using a query. We can use the `JOIN` keyword to join the tables together.

**In the code below we will:**
- Use the `JOIN` keyword to join the `students` and `awards` tables together and get:
    - The first name and last name of the student with the award name
    - The award name for any award won by a student with the first name 'Roger' (or whatever else you want to search for)
    - The last_name, age, and award_name for any award with an award date of today

In [20]:
with duckdb.connect('../data/school.db') as con:
    # Join the awards table with the students table to get the student's name.
    con.sql('''
        SELECT students.first_name, students.last_name, awards.award_name
        FROM students
        JOIN awards
        ON students.student_id = awards.student_id;
    ''').show()
    # Awards for Roger (Change the name in quotes for a different student)
    con.sql('''
        SELECT awards.award_name
        FROM students
        JOIN awards
        ON students.student_id = awards.student_id
        WHERE students.first_name = 'Roger';
    ''').show()
    # Awards from today. Note we used a different (equivalent) syntax for the join this time
    con.sql('''
            SELECT students.last_name, students.age, awards.award_name
            FROM STUDENTS
            JOIN AWARDS USING (student_id)
            WHERE award_date = CURRENT_DATE; 
            ''').show()

┌────────────┬───────────┬──────────────────────────┐
│ first_name │ last_name │        award_name        │
│  varchar   │  varchar  │         varchar          │
├────────────┼───────────┼──────────────────────────┤
│ Roger      │ Rabbit    │ 2024 English Award       │
│ Charlie    │ Bucket    │ 2024 School Spirit Award │
│ Charlie    │ Bucket    │ 2024 Maths Award         │
└────────────┴───────────┴──────────────────────────┘

┌────────────────────┐
│     award_name     │
│      varchar       │
├────────────────────┤
│ 2024 English Award │
└────────────────────┘

┌───────────┬───────┬──────────────────────────┐
│ last_name │  age  │        award_name        │
│  varchar  │ int32 │         varchar          │
├───────────┼───────┼──────────────────────────┤
│ Rabbit    │    18 │ 2024 English Award       │
│ Bucket    │    17 │ 2024 School Spirit Award │
└───────────┴───────┴──────────────────────────┘



## Outer Joins
An outer join returns rows from both tables, joining them where possible. If there is no match, the result is NULL.

Outer joins can be left, right, or full:
- Left outer join: returns all rows from the left (first) table, and the matched rows from the right (second) table
- Right outer join: returns all rows from the right table, and the matched rows from the left table
- Full outer join: returns all rows - matching whenever there is one.

**In the code below we will:**
- Show an example of a full outer join - returning all students details and the details of any awards they have won. Where a student has not won an award, the award details will be NULL. If they have won more than one award, there will be multiple rows for that student.
- Show an example of a right outer join - returning all award details and the details of the student who won the award. Because we have a foreign key on student_id, all awards should have a student_id that matches a student in the students table.


In [25]:
with duckdb.connect('../data/school.db') as con:
    # Outer join the students and awards tables to see all students and their awards.
    con.sql('''
        SELECT * FROM students
        FULL OUTER JOIN  awards
        ON students.student_id = awards.student_id;
    ''').show()
    
    con.sql('''
            SELECT student_id, first_name, award_name from students
            RIGHT JOIN awards
            using(student_id);
            ''').show()

┌────────────┬────────────┬───────────┬───────┬────────────┬──────────┬────────────┬──────────────────────────┬────────────┐
│ student_id │ first_name │ last_name │  age  │ year_level │ award_id │ student_id │        award_name        │ award_date │
│   int32    │  varchar   │  varchar  │ int32 │   int32    │  int32   │   int32    │         varchar          │    date    │
├────────────┼────────────┼───────────┼───────┼────────────┼──────────┼────────────┼──────────────────────────┼────────────┤
│          1 │ Roger      │ Rabbit    │    18 │         11 │        1 │          1 │ 2024 English Award       │ 2024-09-20 │
│        100 │ Charlie    │ Bucket    │    17 │         11 │        3 │        100 │ 2024 School Spirit Award │ 2024-09-20 │
│        100 │ Charlie    │ Bucket    │    17 │         11 │        2 │        100 │ 2024 Maths Award         │ 2024-06-01 │
│          2 │ Jessica    │ Rabbit    │    17 │         10 │     NULL │       NULL │ NULL                     │ NULL       │


## Self Joins ##
A self join is a join of a table with itself. This can be useful when you want to compare rows within a table. 

We do this by giving the table an alias (a temporary name) and then joining the table to itself using that alias.

**In the code below we will:**
- Join the `students` table to itself to find students who have the same first name. We will return the first names and last name of the students who have the same last name.
- *Notes*: 
    - we have an extra check to make sure we don't select the same row from students in s1 and in s2
    - We use s1.student_id < s2.student_id so that they are different students. We could have used <> for not equal to, but that would have put each pair of students in twice. 
    - After s1.last_name 

In [29]:
with duckdb.connect('../data/school.db') as con:
    con.sql('''
            SELECT s1.first_name, s1.last_name last_name_1, s2.last_name last_name_2
            FROM students s1, students s2
            WHERE s1.first_name = s2.first_name
            AND s1.student_id < s2.student_id;
            ''').show()

┌────────────┬─────────────┬─────────────┐
│ first_name │ last_name_1 │ last_name_2 │
│  varchar   │   varchar   │   varchar   │
├────────────┼─────────────┼─────────────┤
│ Roger      │ Rabbit      │ Ramjet      │
└────────────┴─────────────┴─────────────┘



# Your Turn #
- Create a table called `subjects` with the following columns (identifying appropriate data types yourself):
    - `subject_id` - a unique identifier for each subject. This is a primary key so you will need to create a sequence to automatically generate this
    - `year_level` - the year level the subject is taught in (Eg. 7 for year 7., etc.)
    - `subject_name` - the name of the subject (Eg. English, Maths, Data Analytics, etc.)

- Create a table called `classes` with the following columns:
    - `class_id` - a unique identifier for each class. This is a primary key so you will need to create a sequence to automatically generate this
    - `course_name` - the name of the class (eg. Year 7 Maths B, Year 8 English A, etc.)
    - `teacher_code` - the three letter initials for the teacher of the class. This can eventually link to a `teachers` table, but for now just make it a text field
    - `subject_id` - a foreign key that links to the `subjects` table. 

- Add some rows to the `subjects` and `classes` tables. Be sure to link subjects and classes that make sense (ie - Year 7 Maths B should not be a class of the Year 10 Art subject)

- Run a query to join the two tables and check the output.

In [ ]:
# Create the sequence (for the subject_id) and then create subjects table. 
# (Use CREATE OR REPLACE to avoid errors if the table already exists.)

In [ ]:
# Insert multiple rows into the subjects table. We are using INSERT INTO to add new rows to the table.

In [ ]:
# Create the sequence (for class_id) and then create the classes table. Be sure to include the foreign key constraint. 
# (Use CREATE OR REPLACE TABLE to avoid errors if the table already exists.)

In [ ]:
# Insert rows into the classes table. Be sure to match the rows to the existing subjects.


## All together ##
We now want to bring it all together. We need a link between classes and students. We will create a `class_enrolments` table that will link students to classes. This is an intermediate or junction table that will allow us to link students to classes in a many-to-many relationship. If your table is set up correctly, the final query should correctly match students all the way through to the subjects they are studying.

**In the code below you will:**
- Create a `class_enrolments` table with the following columns:
    - `student_id` - a foreign key that links to the `students` table
    - `class_id` - a foreign key that links to the `classes` table
- Add entries to the `class_enrolments` table. You will need to know the `student_id` and `class_id` to do this.
- Run the query to match students to the classes they are enrolled in and the relevant subject for that class.


In [ ]:
# Create the class_enrolments table. Be sure to include the foreign key constraints on both columns. 
# (Use CREATE OR REPLACE TABLE to avoid errors if the table already exists.)

In [ ]:
# Insert rows into the class_enrolments table. Be sure to match the rows to the existing students and classes.

In [ ]:
# If you have been successful, this query should work a treat.
with duckdb.connect('../data/school.db') as con:
    con.sql('''
        SELECT students.first_name, students.last_name, classes.class_name, classes.teacher_name, subjects.subject_name
        FROM students
        JOIN class_enrolments
        ON students.student_id = class_enrolments.student_id
        JOIN classes
        ON class_enrolments.class_id = classes.class_id
        JOIN subjects
        ON classes.subject_id = subjects.subject_id;
    ''').show()